In [50]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.interpolate as spi
import scipy.sparse as sp
import scipy.linalg as sla
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# import pandas as pd
import math
%matplotlib inline
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus'] = False

np.random.seed(1031)
dt_hex = '#2B4750'    # dark teal,  RGB = 43,71,80
r_hex = '#DC2624'     # red,        RGB = 220,38,36
g_hex = '#649E7D'     # green,      RGB = 100,158,125
tl_hex = '#45A0A2'    # teal,       RGB = 69,160,162
tn_hex = '#C89F91'    # tan,        RGB = 200,159,145

## <font color='blue' face='微软雅黑'>欧式期权回顾</font>

在 **BS** 模型下，原生资产 (以股票价格为例) 的随机微分方程如下：

<br/>
<font color='blue'>
\begin{equation}
\frac{dS(t)}{S(t)} = (r-q)dt+\sigma dW(t)
\end{equation}
</font>
    
其中

- $S(t)$ = 资产在时点 $t$ 的值
- $r$ = 常数型瞬时利率
- $q$ = 常数型瞬时红利率
- $\sigma$ = 常数型瞬时波动率
- $W(t)$ = 布朗运动

根据 **BS** 公式，欧式期权的解析解为

<br/>
<font color='blue'>
\begin{equation}
\begin{aligned}
V = e^{-rT} \cdot E\left[ \left[ \omega\left(S_T-K\right)\right]^+\right]
= \omega\cdot\left[e^{-qT}S_0\Phi(\omega\cdot d_+) - e^{-rT}K\Phi(\omega\cdot d_{-})\right] 
\end{aligned}
\end{equation}
</font>
    
其中

<br/>
<font color='blue'>
\begin{equation}
d_{\pm} = \frac{1}{\sigma\sqrt{T}}\ln\left(\frac{S_0e^{(r-q)T}}{K}\right)\pm\frac{\sigma\sqrt{T}}{2}
\end{equation}
</font>

看涨期权对应 $\omega = 1$，看跌期权对应 $\omega = -1$。

In [51]:
def blackscholes( S0=100, K=100, r=0.025, q=0.00, T=231 / 360, sigma=0.2, omega=1 ):
    discount = np.exp(-r*T)
    forward = S0*np.exp((r-q)*T)
    moneyness = np.log(forward/K)
    vol_sqrt_T = sigma*np.sqrt(T)
    
    d1 = moneyness / vol_sqrt_T + 0.5*vol_sqrt_T
    d2 = d1 - vol_sqrt_T
    
    V = omega * discount * (forward*norm.cdf(omega*d1) - K*norm.cdf(omega*d2))
    return V

考虑一个看跌期权，假设股价 S = 50，行权价格 K = 60，利率为 3%，红利率为 1%，期限为 1 年，波动率为 40%，它的特征如下表所示：

| 属性      | 符号        |值 |
|-----------|:------------:|:---:|
| 股票价格 | $S$          |50 |
| 行权价格 | $K$        |60 |
| 连续利率  | $r$        |3% |
| 连续红利率  | $q$        |1% |
| 波动率  | $\sigma$        |40% |
| 到期年限  | $T$        |1 |

带入写好的 `blackscholes` 函数来计算期权的价值。

In [52]:
(S0, K, r, q, T, sigma, omega) = (100, 100, 0.025, 0.00, 231 / 360, 0.2, -1)
blackscholes(S0, K, r, q, T, sigma, omega)

5.569724706320075

### <font color='black' face='微软雅黑'>有限差分</font>

In [53]:
(S, K, r, q, T, sigma, option_type) = (100, 100, 0.025, 0.00, 231 / 360, 0.2, 'put')
(Smin, Smax, Ns, Nt) = (0, 4*np.maximum(S,K), 200, 200)

In [54]:
class OptionPricingMethod():
    
    def __init__(self, S, K, r, q, T, sigma, option_type):
        self.S = S
        self.K = K
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.option_type = option_type
        self.is_call = (option_type[0].lower()=='c')
        self.omega = 1 if self.is_call else -1

In [55]:
class FiniteDifference(OptionPricingMethod):
    
    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt):
        super().__init__(S, K, r, q, T, sigma, option_type)
        self.Smin = Smin
        self.Smax = Smax
        self.Ns = int(Ns)
        self.Nt = int(Nt)
        self.dS = (Smax-Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(Smin, Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
        
    def _set_terminal_condition_(self):
        self.grid[:, -1] = np.maximum(self.omega*(self.Svec - self.K), 0)
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec;     
        DFq = np.exp(-q*tau)
        DFr = np.exp(-r*tau)

        self.grid[0,  :] = np.maximum(self.omega*(self.Svec[0]*DFq - self.K*DFr), 0)
        self.grid[-1, :] = np.maximum(self.omega*(self.Svec[-1]*DFq - self.K*DFr), 0)        
        
    def _set_coefficient__(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
        #return np.interp(self.S, self.Svec, self.grid[:,0])
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient__()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

### <font color='black' face='微软雅黑'>克兰克尼克尔森法 </font>

In [56]:
class CrankNicolsonEu(FiniteDifference):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [57]:
# (theta, alpha, epsilon) = (0.5, 1.5, 1e-6)
euro_opt = CrankNicolsonEu(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt)
print(euro_opt.price())

5.557399371378562


## <font color='blue' face='微软雅黑'>美式期权</font>
### <font color='black' face='微软雅黑'>迭代法</font>

In [58]:
class SOR(FiniteDifference):

    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon):
        super().__init__(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt)
        self.theta = theta
        self.alpha = alpha
        self.epsilon = epsilon
        self.max_iter = 10*Nt
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
    
    def _solve_(self):           
        w = self.alpha
        thedt = self.theta * self.dt
        payoff = self.grid[1:-1, -1]
        m = len(payoff)
        pastval = payoff.copy()
        
        for j in reversed(np.arange(self.Nt)):
            counter = 0
            noBreak = 1
            newval = pastval.copy()
            
            z = self.M1.dot(pastval)
            
            z[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            z[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            while noBreak:
                counter += 1
                oldval = newval.copy()
                newval[0] = np.maximum( payoff[0], oldval[0] + w/(1-thedt*self.c[0]) \
                                       *( z[0] - (1-thedt*self.c[0])*oldval[0] \
                                         + thedt*self.u[0]*oldval[1]) )
                for k in np.arange(1,m-1):
                    newval[k] = np.maximum( payoff[k], oldval[k] + w/(1-thedt*self.c[k]) \
                                           *( z[k] + thedt*self.l[k]*newval[k-1] \
                                             - (1-thedt*self.c[k])*oldval[k] \
                                             + thedt*self.u[k]*oldval[k+1]) )
        
                newval[m-1] = np.maximum( payoff[m-1], oldval[m-1] + w/(1-thedt*self.c[m-1]) \
                                         *( z[m-1] + thedt*self.l[m-1]*newval[m-2] \
                                           - (1-thedt*self.c[m-1])*oldval[m-1]) )
        
                noBreak = SOR.trigger( oldval, newval, self.epsilon, counter, self.max_iter )
                
            pastval = newval.copy()
            self.grid[1:-1, j] = pastval
      
    @staticmethod
    def trigger( oldval, newval, tol, counter, maxIteration ):
        noBreak = 1
        if np.max( np.abs(newval-oldval)/np.maximum(1,np.abs(newval)) ) <= tol:
            noBreak = 0
        elif counter > maxIteration:
            print('结果可能不收敛。')
            noBreak = 0
        return noBreak

In [59]:
(theta, alpha, epsilon) = (0.5, 1.5, 1e-6)
amer_opt = SOR(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon)
print(amer_opt.price())

5.692073676500009


### <font color='black' face='微软雅黑'>惩罚法</font>

In [60]:
class PM(FiniteDifference):

    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, lbd, epsilon):
        super().__init__(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt)
        self.theta = theta
        self.lbd = lbd
        self.epsilon = epsilon
        self.max_iter = 10*Nt
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1], format='csc')
        self.I = sp.eye(self.Ns-1)
    
    def _solve_(self):           
        (theta, dt) = (self.theta, self.dt)
        payoff = self.grid[1:-1, -1]
        pastval = payoff.copy()
        G = payoff.copy()
        
        for j in reversed(np.arange(self.Nt)):
            counter = 0
            noBreak = 1
            newval = pastval.copy()
            
            while noBreak:
                counter += 1
                oldval = newval.copy()
                D = sp.diags( (G > (1-theta)*pastval + theta*newval).astype(int), format='csc' )
                z = (self.I + (1-theta)*dt*(self.A - self.lbd*D))*pastval + dt*self.lbd*D*G
                
                z[0] += theta*self.l[0]*dt*self.grid[0, j] \
                 + (1-theta)*self.l[0]*dt*self.grid[0, j+1] 
                z[-1] += theta*self.u[-1]*dt*self.grid[-1, j] \
                  + (1-theta)*self.u[-1]*dt*self.grid[-1, j+1] 
                                
                M = self.I - theta*dt*(self.A - self.lbd*D)
                newval = spsolve(M,z)
        
                noBreak = PM.trigger( oldval, newval, self.epsilon, counter, self.max_iter )
            
            pastval = newval.copy()
            self.grid[1:-1, j] = pastval
    
    @staticmethod
    def trigger( oldval, newval, tol, counter, maxIteration ):
        noBreak = 1
        if np.max( np.abs(newval-oldval)/np.maximum(1,np.abs(newval)) ) <= tol:
            noBreak = 0
        elif counter > maxIteration:
            print('结果可能不收敛。')
            noBreak = 0
        return noBreak

In [61]:
(theta, lbd, epsilon) = (0.5, 1e6, 1e-6)
amer_opt = PM(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, lbd, epsilon)
print(amer_opt.price())

5.6920618221348676


In [65]:
class american_trinominal_model:

    def __init__(self,S=1,T=60,K=1,r=0,q=0,sigma=.1, call = True):
        self.S = S
        self.T = T
        self.K = K
        self.r = r
        self.q = q
        self.sigma = sigma
        self.call  = call


    def price(self):
        dt = 1./360
        #number of timesteps
        N = int(self.T/dt)
        #mu is r-q - (sigma^2)/2
        mu = self.r-self.q-(self.sigma**2)/2.0
        #set sigma max for stability requirements
        smax = 2 * abs(mu) * dt**.5
        smax = max(smax, self.sigma * (2**.5))
        if smax ==0:
            return -9999
        #set up arrays to keep track of steps
        #dimension M
        M = int(5 * (N**.5))
        C_ = np.empty(2*M+1, dtype=np.float64)
        pC_ = np.empty(2*M+1, dtype=np.float64)
        S_ = np.empty(2*M+1, dtype=np.float64,)
        #set probs up, down, and same
        p = float(0.5 * (self.sigma**2) )/ (smax **2)
        p_u = p + 0.5 * mu * dt**.5 / float(smax)
        p_m = 1 - 2 * p
        p_d = p - 0.5 * mu * dt**.5 / float(smax)
        #init payoff
        D = 1.0 / (1 + self.r * dt)
        E = math.exp(smax * dt**.5)
        
        for j in range(0,len(S_)):
            if j ==0:
                S_[j] = self.S * math.exp(-M * smax * dt**.5)
            else:
                S_[j] = S_[j - 1] * E
            if self.call ==True:
                C_[j] = max(S_[j] - self.K, 0)
            else:
                C_[j] = max(self.K-S_[j], 0)

        for k in range(0,N):
            for j in range(1,2 * M):
                pC_[j] = (p_u * C_[j + 1] + p_m * C_[j] + p_d * C_[j - 1])*D
            #set boundaries
            pC_[0] = 2 * pC_[1] - pC_[2]
            pC_[2 * M] = 2 * pC_[2 * M -1] - pC_[2 * M - 2]
            
            for n in range(0,2 * M+1):
                if self.call ==True:
                    C_[n] = max(pC_[n],max(S_[n]-self.K,0))
                else:
                    C_[n] = max(pC_[n],max(self.K-S_[n],0))
        ret = C_[M]
        return ret

american_trinominal_model(S=S, K=K, r=r, q=q, T=T, sigma=sigma, call=False).price()

5.703702593211633

In [63]:


def monte_carlo_european_put(S0, K, r, sigma, T, num_simulations):
    np.random.seed(42)  # For reproducibility
    num_timesteps = 10
    dt = T / num_timesteps

    total_payoff = 0
    for i in range(num_simulations):
        S = S0
        for j in range(num_timesteps):
            z = np.random.standard_normal()
            S *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

        total_payoff += max(K - S, 0)

    price = np.exp(-r * T) * total_payoff / num_simulations
    return price

# Parameters from part (a)
S0 = 100
K = 100
r = 0.025
sigma = 0.20


num_simulations = 10000
monte_carlo_price = monte_carlo_european_put(S0, K, r, sigma, T, num_simulations)

print(f"Monte Carlo European Put price: {monte_carlo_price}")

Monte Carlo European Put price: 5.667516673741924
